In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

In [4]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

In [5]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

In [6]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

In [7]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

In [8]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

In [9]:
##transform into 1d label array 
def transformArrLabel(X_label, Y_label):
    X_transformed = []
    Y_transformed = []
    for i in range(len(X_label)):
        X_transformed.append(X_label[i][0])
    for i in range(len(Y_label)):
        Y_transformed.append(Y_label[i][0])
    return np.array(X_transformed), np.array(Y_transformed)

In [10]:
k = 15
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)

In [12]:
k_range = range(1,16)

arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for k in k_range:
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        train_l, test_l = transformArrLabel(train_l, test_l)
        knn = KNeighborsClassifier(n_neighbors = k, algorithm='brute')
        knn.fit(train_f, train_l)
        y_pred = knn.predict(test_f)
        acc = metrics.accuracy_score(test_l, y_pred)
        arr_acc[i-1] = acc
    mean, std = evaluation(arr_acc)
    arr_mean_acc[k-1] = mean
    arr_std_acc[k-1] = std
    print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(k,mean,std))

K-1 Mean : 0.9512158054711246 with Standar Deviation : 0.008140280693521029.
K-2 Mean : 0.9465045592705167 with Standar Deviation : 0.011101509038145689.
K-3 Mean : 0.9554711246200608 with Standar Deviation : 0.00778050196727673.
K-4 Mean : 0.9519756838905776 with Standar Deviation : 0.00939794822719779.
K-5 Mean : 0.9588145896656535 with Standar Deviation : 0.007490104273769967.
K-6 Mean : 0.9527355623100304 with Standar Deviation : 0.007520877222892762.
K-7 Mean : 0.9572948328267478 with Standar Deviation : 0.007880785509096763.
K-8 Mean : 0.9542553191489361 with Standar Deviation : 0.009836243845157702.
K-9 Mean : 0.9566869300911854 with Standar Deviation : 0.00904861741434905.
K-10 Mean : 0.9531914893617021 with Standar Deviation : 0.008262782503992816.
K-11 Mean : 0.9557750759878421 with Standar Deviation : 0.009765546399037902.
K-12 Mean : 0.9533434650455928 with Standar Deviation : 0.010375610449056692.
K-13 Mean : 0.9548632218844986 with Standar Deviation : 0.010574057013251629